# Fase 4: Modelagem e Validação (EAP 4.0)

Este notebook implementa a Fase 4 do projeto de previsão de tendência do IBOVESPA, focando no treinamento dos modelos, na sua avaliação rigorosa e na validação da robustez dos resultados.

**Autor:** Projeto Tech Challenge 2  
**Data:** 2025-01-24  
**Referência:** EAP.md e Steering.md

## Importações e Configurações Iniciais

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('default')
sns.set_palette("husl")

print("✓ Bibliotecas importadas com sucesso")

## Carregamento dos Dados Preparados

**Nota:** Substitua esta célula pelo carregamento dos seus dados preparados da Fase 3.

In [ ]:
# SUBSTITUA ESTA CÉLULA PELO CARREGAMENTO DOS SEUS DADOS
# Exemplo de como os dados devem estar estruturados:
# dados_preparados = {
#     'X_train_scaled': X_train_scaled,
#     'y_train': y_train,
#     'X_test_scaled': X_test_scaled,
#     'y_test': y_test
# }

# Para demonstração, vamos criar dados fictícios
# REMOVA ESTA SEÇÃO E USE SEUS DADOS REAIS
np.random.seed(42)
n_samples_train = 1000
n_samples_test = 200
n_features = 10

X_train_scaled = pd.DataFrame(
    np.random.randn(n_samples_train, n_features),
    columns=[f'feature_{i}' for i in range(n_features)]
)
y_train = pd.Series(np.random.choice([0, 1], n_samples_train, p=[0.6, 0.4]))

X_test_scaled = pd.DataFrame(
    np.random.randn(n_samples_test, n_features),
    columns=[f'feature_{i}' for i in range(n_features)]
)
y_test = pd.Series(np.random.choice([0, 1], n_samples_test, p=[0.6, 0.4]))

dados_preparados = {
    'X_train_scaled': X_train_scaled,
    'y_train': y_train,
    'X_test_scaled': X_test_scaled,
    'y_test': y_test
}

print("✓ Dados carregados")
print(f"✓ Amostras de treino: {len(X_train_scaled)}")
print(f"✓ Amostras de teste: {len(X_test_scaled)}")
print(f"✓ Número de features: {X_train_scaled.shape[1]}")

## 4.1 Treinamento do Modelo Baseline (Regressão Logística)

### 4.1.1 - Instanciar e treinar um modelo de Regressão Logística
### 4.1.2 - Realizar previsões no conjunto de teste

In [ ]:
print("=== TREINAMENTO DO MODELO BASELINE (REGRESSÃO LOGÍSTICA) ===")

# Extrai dados
X_train = dados_preparados['X_train_scaled']
y_train = dados_preparados['y_train']
X_test = dados_preparados['X_test_scaled']
y_test = dados_preparados['y_test']

# 4.1.1 - Instancia e treina Regressão Logística
modelo_baseline = LogisticRegression(
    random_state=42,
    max_iter=1000,
    solver='liblinear'  # Adequado para datasets pequenos/médios
)

modelo_baseline.fit(X_train, y_train)

# 4.1.2 - Realiza previsões
y_pred_baseline = modelo_baseline.predict(X_test)
y_pred_proba_baseline = modelo_baseline.predict_proba(X_test)[:, 1]

# Armazena resultados
resultados_baseline = {
    'modelo': modelo_baseline,
    'y_pred': y_pred_baseline,
    'y_pred_proba': y_pred_proba_baseline,
    'y_true': y_test
}

print(f"✓ Modelo de Regressão Logística treinado")
print(f"✓ Previsões realizadas no conjunto de teste")
print(f"✓ Amostras de treino: {len(X_train)}")
print(f"✓ Amostras de teste: {len(X_test)}")

## 4.2 Treinamento do Modelo Principal (XGBoost)

### 4.2.1 - Instanciar um XGBClassifier
### 4.2.2 - Configurar scale_pos_weight para desbalanceamento
### 4.2.3 - Treinar o modelo
### 4.2.4 - Realizar previsões

In [ ]:
print("=== TREINAMENTO DO MODELO PRINCIPAL (XGBOOST) ===")

# Calcula scale_pos_weight para balanceamento
contagem_classes = y_train.value_counts()
scale_pos_weight = contagem_classes[0] / contagem_classes[1] if 1 in contagem_classes else 1

print(f"✓ Scale pos weight calculado: {scale_pos_weight:.4f}")

# 4.2.1 e 4.2.2 - Instancia XGBClassifier com configurações
modelo_xgboost = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='logloss'
)

# 4.2.3 - Treina o modelo
modelo_xgboost.fit(X_train, y_train)

# 4.2.4 - Realiza previsões
y_pred_xgb = modelo_xgboost.predict(X_test)
y_pred_proba_xgb = modelo_xgboost.predict_proba(X_test)[:, 1]

# Armazena resultados
resultados_xgboost = {
    'modelo': modelo_xgboost,
    'y_pred': y_pred_xgb,
    'y_pred_proba': y_pred_proba_xgb,
    'y_true': y_test,
    'feature_importance': modelo_xgboost.feature_importances_
}

print(f"✓ Modelo XGBoost treinado")
print(f"✓ Previsões realizadas no conjunto de teste")
print(f"✓ Hiperparâmetros configurados para robustez")

## 4.3 Avaliação de Métricas de Desempenho

### 4.3.1 - Calcular e analisar métricas para ambos os modelos
- Matriz de Confusão
- Precisão (Precision)
- Revocação (Recall)
- F1-Score

In [ ]:
def avaliar_metricas_desempenho(resultados, nome_modelo):
    """
    Calcula e exibe métricas de desempenho para um modelo.
    """
    y_true = resultados['y_true']
    y_pred = resultados['y_pred']
    
    print(f"\n=== AVALIAÇÃO DE MÉTRICAS - {nome_modelo.upper()} ===")
    
    # Matriz de Confusão
    cm = confusion_matrix(y_true, y_pred)
    
    # Métricas
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')
    accuracy = (y_pred == y_true).mean()
    
    # Relatório detalhado
    report = classification_report(y_true, y_pred, output_dict=True)
    
    metricas = {
        'matriz_confusao': cm,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy,
        'classification_report': report
    }
    
    # Exibe resultados
    print(f"✓ Matriz de Confusão:")
    print(f"   [[TN={cm[0,0]}, FP={cm[0,1]}],")
    print(f"    [FN={cm[1,0]}, TP={cm[1,1]}]]")
    
    print(f"\n✓ Métricas de Desempenho:")
    print(f"   - Acurácia: {accuracy:.4f}")
    print(f"   - Precisão: {precision:.4f}")
    print(f"   - Recall: {recall:.4f}")
    print(f"   - F1-Score: {f1:.4f}")
    
    # Interpretação das métricas
    print(f"\n✓ Interpretação:")
    print(f"   - Precisão: {precision:.2%} das previsões de 'alta' estão corretas")
    print(f"   - Recall: {recall:.2%} dos dias de 'alta' foram identificados")
    print(f"   - F1-Score: {f1:.4f} (média harmônica de precisão e recall)")
    
    return metricas

# Avalia modelo baseline
metricas_baseline = avaliar_metricas_desempenho(resultados_baseline, "Regressão Logística")

In [ ]:
# Avalia modelo XGBoost
metricas_xgboost = avaliar_metricas_desempenho(resultados_xgboost, "XGBoost")

## Visualização das Matrizes de Confusão

In [ ]:
def plotar_matriz_confusao(resultados, nome_modelo, salvar_grafico=False):
    """
    Plota a matriz de confusão de forma visual.
    """
    y_true = resultados['y_true']
    y_pred = resultados['y_pred']
    
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
               xticklabels=['Baixa (0)', 'Alta (1)'],
               yticklabels=['Baixa (0)', 'Alta (1)'])
    plt.title(f'Matriz de Confusão - {nome_modelo}', fontsize=14, fontweight='bold')
    plt.xlabel('Predição', fontsize=12)
    plt.ylabel('Valor Real', fontsize=12)
    
    if salvar_grafico:
        plt.savefig(f'Tech Challenge 2/final/matriz_confusao_{nome_modelo.lower().replace(" ", "_")}.png', 
                   dpi=300, bbox_inches='tight')
        print(f"✓ Matriz de confusão salva para {nome_modelo}")
    
    plt.show()

# Plota matrizes de confusão
plotar_matriz_confusao(resultados_baseline, "Regressão Logística")
plotar_matriz_confusao(resultados_xgboost, "XGBoost")

## 4.4 Validação Walk-Forward

### 4.4.1 - Implementar validação walk-forward com 3 dobras
### 4.4.2 - Treinar XGBoost para cada dobra
### 4.4.3 - Coletar métricas de cada dobra
### 4.4.4 - Calcular média e desvio padrão

In [ ]:
def validacao_walk_forward(dados_preparados, n_splits=3):
    """
    Implementa validação walk-forward simplificada.
    """
    print(f"=== VALIDAÇÃO WALK-FORWARD ({n_splits} DOBRAS) ===")
    
    # Dados completos (treino + teste)
    X_completo = pd.concat([
        dados_preparados['X_train_scaled'], 
        dados_preparados['X_test_scaled']
    ])
    y_completo = pd.concat([
        dados_preparados['y_train'], 
        dados_preparados['y_test']
    ])
    
    # Configuração do TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    metricas_dobras = []
    
    for i, (train_idx, test_idx) in enumerate(tscv.split(X_completo)):
        print(f"\n--- Dobra {i+1}/{n_splits} ---")
        
        # Dados da dobra
        X_train_fold = X_completo.iloc[train_idx]
        y_train_fold = y_completo.iloc[train_idx]
        X_test_fold = X_completo.iloc[test_idx]
        y_test_fold = y_completo.iloc[test_idx]
        
        # Calcula scale_pos_weight para a dobra
        contagem_classes = y_train_fold.value_counts()
        scale_pos_weight = contagem_classes[0] / contagem_classes[1] if 1 in contagem_classes else 1
        
        # Treina modelo XGBoost para a dobra
        modelo_fold = xgb.XGBClassifier(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            scale_pos_weight=scale_pos_weight,
            random_state=42,
            eval_metric='logloss'
        )
        
        modelo_fold.fit(X_train_fold, y_train_fold)
        
        # Previsões
        y_pred_fold = modelo_fold.predict(X_test_fold)
        
        # Métricas da dobra
        precision_fold = precision_score(y_test_fold, y_pred_fold, average='binary')
        recall_fold = recall_score(y_test_fold, y_pred_fold, average='binary')
        f1_fold = f1_score(y_test_fold, y_pred_fold, average='binary')
        accuracy_fold = (y_pred_fold == y_test_fold).mean()
        
        metricas_fold = {
            'dobra': i+1,
            'precision': precision_fold,
            'recall': recall_fold,
            'f1_score': f1_fold,
            'accuracy': accuracy_fold,
            'periodo_treino': f"{X_train_fold.index.min()} até {X_train_fold.index.max()}",
            'periodo_teste': f"{X_test_fold.index.min()} até {X_test_fold.index.max()}",
            'amostras_treino': len(X_train_fold),
            'amostras_teste': len(X_test_fold)
        }
        
        metricas_dobras.append(metricas_fold)
        
        print(f"   Período treino: {metricas_fold['periodo_treino']}")
        print(f"   Período teste: {metricas_fold['periodo_teste']}")
        print(f"   F1-Score: {f1_fold:.4f}")
        print(f"   Precisão: {precision_fold:.4f}")
        print(f"   Recall: {recall_fold:.4f}")
    
    return metricas_dobras

# Executa validação walk-forward
metricas_dobras = validacao_walk_forward(dados_preparados, n_splits=3)

In [ ]:
# Calcula estatísticas agregadas
df_metricas = pd.DataFrame(metricas_dobras)

estatisticas_agregadas = {
    'precision_media': df_metricas['precision'].mean(),
    'precision_std': df_metricas['precision'].std(),
    'recall_media': df_metricas['recall'].mean(),
    'recall_std': df_metricas['recall'].std(),
    'f1_score_media': df_metricas['f1_score'].mean(),
    'f1_score_std': df_metricas['f1_score'].std(),
    'accuracy_media': df_metricas['accuracy'].mean(),
    'accuracy_std': df_metricas['accuracy'].std()
}

resultados_walk_forward = {
    'metricas_por_dobra': metricas_dobras,
    'estatisticas_agregadas': estatisticas_agregadas,
    'dataframe_metricas': df_metricas
}

print(f"\n=== RESULTADOS AGREGADOS DA VALIDAÇÃO WALK-FORWARD ===")
print(f"✓ F1-Score: {estatisticas_agregadas['f1_score_media']:.4f} ± {estatisticas_agregadas['f1_score_std']:.4f}")
print(f"✓ Precisão: {estatisticas_agregadas['precision_media']:.4f} ± {estatisticas_agregadas['precision_std']:.4f}")
print(f"✓ Recall: {estatisticas_agregadas['recall_media']:.4f} ± {estatisticas_agregadas['recall_std']:.4f}")
print(f"✓ Acurácia: {estatisticas_agregadas['accuracy_media']:.4f} ± {estatisticas_agregadas['accuracy_std']:.4f}")

# Exibe tabela com resultados por dobra
print("\n=== MÉTRICAS POR DOBRA ===")
display(df_metricas[['dobra', 'precision', 'recall', 'f1_score', 'accuracy', 'amostras_treino', 'amostras_teste']])

## Visualização dos Resultados da Validação Walk-Forward

In [ ]:
# Gráfico de barras com as métricas por dobra
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Métricas por Dobra - Validação Walk-Forward', fontsize=16, fontweight='bold')

metricas_plot = ['precision', 'recall', 'f1_score', 'accuracy']
titulos = ['Precisão', 'Recall', 'F1-Score', 'Acurácia']

for i, (metrica, titulo) in enumerate(zip(metricas_plot, titulos)):
    ax = axes[i//2, i%2]
    
    bars = ax.bar(df_metricas['dobra'], df_metricas[metrica], 
                  color=sns.color_palette("husl", len(df_metricas)), alpha=0.8)
    
    # Adiciona valores nas barras
    for bar, valor in zip(bars, df_metricas[metrica]):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{valor:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # Linha da média
    media = df_metricas[metrica].mean()
    ax.axhline(y=media, color='red', linestyle='--', alpha=0.7, 
               label=f'Média: {media:.3f}')
    
    ax.set_title(titulo, fontsize=12, fontweight='bold')
    ax.set_xlabel('Dobra')
    ax.set_ylabel(titulo)
    ax.set_ylim(0, 1)
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Comparação Final dos Modelos

In [ ]:
# Tabela comparativa final
comparacao_final = pd.DataFrame({
    'Modelo': ['Regressão Logística', 'XGBoost', 'XGBoost (Walk-Forward)'],
    'Precisão': [
        metricas_baseline['precision'],
        metricas_xgboost['precision'],
        estatisticas_agregadas['precision_media']
    ],
    'Recall': [
        metricas_baseline['recall'],
        metricas_xgboost['recall'],
        estatisticas_agregadas['recall_media']
    ],
    'F1-Score': [
        metricas_baseline['f1_score'],
        metricas_xgboost['f1_score'],
        estatisticas_agregadas['f1_score_media']
    ],
    'Acurácia': [
        metricas_baseline['accuracy'],
        metricas_xgboost['accuracy'],
        estatisticas_agregadas['accuracy_media']
    ]
})

print("=== COMPARAÇÃO FINAL DOS MODELOS ===")
display(comparacao_final.round(4))

# Identifica o melhor modelo
melhor_f1 = comparacao_final.loc[comparacao_final['F1-Score'].idxmax()]
print(f"\n✓ Melhor modelo por F1-Score: {melhor_f1['Modelo']} (F1: {melhor_f1['F1-Score']:.4f})")

print("\n=== CONCLUSÕES ===")
print("✓ Fase 4 concluída com sucesso")
print("✓ Modelos treinados e avaliados")
print("✓ Validação walk-forward implementada")
print("✓ Métricas de robustez calculadas")
print("✓ Comparação entre modelos realizada")